In [1]:
from shapely import wkt
from shapely.geometry import Point
import geojson
import pandas as pd
import numpy as np
import keplergl
from odps.df import DataFrame
from util import GisTransform
import json
from shapely.geometry import MultiPoint, Polygon
import geopandas
import hashlib
from show_data import create, transfer

gis = GisTransform('wgs84', 'gcj02')

In [8]:
df = pd.read_csv('安庆_20191126.csv')
df = df.rename(columns={'address': 'name'})

house_price_df = df[df['price'] > 0].groupby('name').agg({'avg_price':'mean'}).reset_index()

geo_df = pd.read_csv('aq_poi.csv')

t = pd.merge(house_price_df, geo_df[['name', 'shape']], on='name', how='left')
t = t.dropna()
t['geojson'] = t.apply(lambda item: transfer(item['shape'], item['name']), axis=1)

In [24]:
config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'drkxvhs',
     'type': 'geojson',
     'config': {'dataId': 'heatmap',
      'label': 'heatmap',
      'color': [18, 147, 154],
      'columns': {'geojson': 'shape'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': False,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'y1qq54j',
     'type': 'geojson',
     'config': {'dataId': 'heatmap',
      'label': 'heatmap',
      'color': [221, 178, 124],
      'columns': {'geojson': 'geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [136, 87, 44],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'avg_price', 'type': 'real'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'heatmap': ['name',
       'avg_price']},
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 30.520448469448702,
   'longitude': 117.05966573544289,
   'pitch': 0,
   'zoom': 11.272637349157469,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [25]:
wholemap = keplergl.KeplerGl(height=800)  
wholemap.config = config

wholemap.add_data(data=t, name="heatmap")
wholemap

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'drkxvhs', 'type': …

In [27]:
wholemap.save_to_html(file_name='aq_heatmap.html')

Map saved to aq_heatmap.html!
